In [1]:
import PIL,torch
import matplotlib.pyplot as plt
import torch.nn as nn
import numpy as np
import os
import time
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from torchvision.models import resnet50

In [2]:
from tensorboardX import SummaryWriter


suffix = '%f' % time.time()
log_dir = './log/%s'% suffix
EXPERIMENT = 'resnet50_punish'

datdir = os.path.join(os.getcwd(), 'Skin40')
print(datdir)

/home/jovyan/SYSUDeepLearningSkin/Skin40


In [3]:
class SkinDataset(Dataset):
    folds=5
    def __init__(self, root , num_classes,fold=0,training=False,transform=None):
        self.data_path = []
        self.transform = transform
        if self.transform is None:
            self.transform = transforms.Compose([
                transforms.RandomResizedCrop(size=(224, 224)),
                transforms.RandomRotation(5),
                transforms.ToTensor(),
                transforms.Normalize(mean= [0.6075306,0.49116918 ,0.46066117],std = [0.22603881, 0.21623525, 0.2191065 ])
            ]
            )
        for label in range(num_classes):
            self.data_dir = os.path.join(root,os.listdir(root)[label])
            self.filename = os.listdir(self.data_dir)
            l = len(self.filename)
            inter = l//SkinDataset.folds
            picked = list(range(inter* fold,inter * (fold+1))) if not training else list(range(0,inter*fold))+list(range(inter*(fold+1),l))

            for i in picked:
                file_path = os.path.join(self.data_dir , self.filename[i])
                self.data_path.append((file_path, label))
    
    def __getitem__(self , index):
        ddir , label = self.data_path[index]
        img = Image.open(ddir)
        result = (self.transform(img) , label)
        del img
        return result
    
    def __len__(self):
        return len(self.data_path)

In [4]:
ds = SkinDataset(os.path.join(os.getcwd(),'Skin40'), 40,training=True)

In [5]:
# del training_dataloader
training_dataset = SkinDataset(os.path.join(os.getcwd(),'Skin40'), 40, training=True)
test_dataset = SkinDataset(os.path.join(os.getcwd(),'Skin40'), 40)
training_dataloader = DataLoader(training_dataset , batch_size = 128 , num_workers = 1, shuffle = True)
test_dataloader = DataLoader(test_dataset , batch_size = 128 , num_workers = 1, shuffle = True)
print()
print()

In [6]:
def train(model, train_loader, loss_func, optimizer, device):
    """
    train model using loss_fn and optimizer in an epoch.
    model: CNN networks
    train_loader: a Dataloader object with training data
    loss_func: loss function
    device: train on cpu or gpu device
    """
    total_loss = 0
    # train the model using minibatch
    print(len(train_loader))
    for i, (images, targets) in enumerate(train_loader):
        images = images.to(device)
        targets = targets.to(device)
        # forward
#        print(i)
        outputs = model(images)
        loss = loss_func(outputs, targets)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    print('train ok')
    return total_loss / len(train_loader)

In [7]:
def evaluate(model, val_loader, device):
    """
    model: CNN networks
    val_loader: a Dataloader object with validation data
    device: evaluate on cpu or gpu device
    return classification accuracy of the model on val dataset
    """
    # evaluate the model
    model.eval()
    # context-manager that disabled gradient computation
    with torch.no_grad():
        correct = 0
        total = 0
        
        # record all target && correct prediction by each class
        all_tar = np.zeros((40)).tolist()
        cor_pre =  np.zeros((40)).tolist()

        for i, (images, targets) in enumerate(val_loader):
            # device: cpu or gpu
            images = images.to(device)
            targets = targets.to(device)
            
            outputs = model(images)
            
            # return the maximum value of each row of the input tensor in the 
            # given dimension dim, the second return vale is the index location
            # of each maxium value found(argmax)
            _, predicted = torch.max(outputs.data, dim=1)
            cor_bol_list = (predicted == targets)
            correct += (cor_bol_list).sum().item()
            total += targets.size(0)
            # 
            for i,bl in enumerate( cor_bol_list):
                all_tar[targets[i].cpu().numpy().tolist()] += 1
                if bl:
                    cor_pre[predicted[i].cpu().numpy().tolist()] += 1
        # accuracy of tatal && each class 
        accuracy = correct / total
        each_acc = torch.tensor(cor_pre) / torch.tensor(all_tar)
        return accuracy,each_acc

In [8]:
def fit(model, num_epochs, optimizer, device):
    """
     train and evaluate an classifier num_epochs times.
    n and evaluate an classifier num_epochs times.
    We use optimizer and cross entropy loss to train the model. 
    Args: 
        model: CNN network
        num_epochs: the number of training epochs
        optimizer: optimize the loss function    loss_func.to(device)
    loss_func.to(device)

    """
    # optimizer
    model.to(device)
    loss_func = None
    mask = None
    
    writer = SummaryWriter(log_dir=log_dir + '/%s training loss' % EXPERIMENT)
    for epoch in range(num_epochs):
        print(epoch) 
        if epoch ==0:
            loss_func = nn.CrossEntropyLoss()
        else:
            loss_func = nn.CrossEntropyLoss(weight=mask)
        
        loss_func.to(device)
        # tensorboard
        # train step
        loss = train(model, training_dataloader, loss_func, optimizer, device)
        
        # save model
        torch.save(model.state_dict(), 'model_%s.pth' % EXPERIMENT)
        print('train evaluate')
        accuracy,each_acc = evaluate(model, training_dataloader, device)   
        mask = torch.div(torch.ones_like(each_acc).to(device) - each_acc.to(device),accuracy).to(device) + torch.ones_like(each_acc).to(device)
        loss_func_parm = (mask,device)

        print('train finish')
        writer.add_scalar('training loss', loss, epoch)
        writer.add_scalar('training acc', accuracy, epoch)        
        for i in range(40):
            temp_writer = SummaryWriter(log_dir=log_dir + '/%s' % EXPERIMENT + ' training class %s acc' % str(i))
            temp_writer.add_scalar('training class acc',each_acc[i], epoch)
            temp_writer.close()
            
        # evaluate step
        accuracy,each_acc = evaluate(model, test_dataloader, device) 
        print('eval')
        writer.add_scalar('testing acc', accuracy, epoch)
        for i in range(40):
            temp_writer = SummaryWriter(log_dir=log_dir + '/%s' % EXPERIMENT + 'testing class %s acc' % str(i))
            writer.add_scalar('testing class acc',each_acc[i], epoch)
            temp_writer.close()
    writer.close()


In [9]:
model = resnet50(pretrained=True)
#print(model)

In [10]:
# Hyper-parameters
num_epochs = 10
lr = 0.001
# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)
for parma in model.parameters():
    parma.requires_grad = False
model.fc = torch.nn.Sequential(nn.Linear(model.fc.in_features, 40))
# optimizer
optimizer = torch.optim.SGD(model.fc.parameters(), lr=lr,momentum=0.9)

fit(model, num_epochs, optimizer, device)  #

cuda:0
0
15
train ok
train evaluate
train finish
eval
1
15
train ok
train evaluate
train finish
eval
2
15
train ok
train evaluate
train finish
eval
3
15
train ok
train evaluate
train finish
eval
4
15
train ok
train evaluate
train finish
eval
5
15
train ok
train evaluate
train finish
eval
6
15
train ok
train evaluate
train finish
eval
7
15
train ok
train evaluate
train finish
eval
8
15
train ok
train evaluate
train finish
eval
9
15
train ok
train evaluate
train finish
eval


In [11]:
for i,(dat, tar) in enumerate(test_dataloader):
    print(i ,dat.shape, tar)
    if ( i>30):
        break

0 torch.Size([128, 3, 224, 224]) tensor([23, 11, 21, 14, 17, 37, 38, 24, 25, 20, 30, 26,  8,  0,  1, 38, 32, 15,
         1, 21, 17, 15, 12, 33, 28,  2, 14, 27, 20,  7, 34,  7, 26, 19, 32, 22,
         7, 15, 20, 29,  4,  8, 13,  8, 24,  2, 34, 27, 30, 38, 14,  0,  0, 38,
         1,  6, 25,  2, 25,  6, 36,  9, 27, 37,  3, 23, 31,  2,  7, 10, 18, 35,
        17,  7, 16,  6,  8, 27,  0, 11,  3, 14,  1,  9, 28,  8, 26, 24, 33,  5,
         1,  3, 35, 20,  4, 32,  7, 30, 31, 23, 35, 18, 19, 35, 33,  6,  8, 29,
         4,  4, 38,  7,  7,  8, 37,  1, 21, 29,  9, 23,  7, 13, 37,  1,  3, 27,
        24,  5])
1 torch.Size([128, 3, 224, 224]) tensor([ 1, 18, 37, 18,  5, 36, 11, 13, 21, 12, 13, 35, 38, 11, 12,  9,  5, 16,
        23, 11, 39, 27, 11, 13,  1, 27, 20, 26,  0, 10, 28, 22, 39,  0, 16, 11,
         1, 14, 38, 30, 31, 37, 25, 32, 16, 22,  8, 16, 36, 29,  4, 37, 32,  8,
         0, 21,  3, 16, 36, 21, 19,  6, 14, 25, 19, 27, 30, 13, 28,  9, 17, 28,
        36,  8, 22, 16, 20,  0, 15, 3